In [1]:
import torch
from torch import nn
from torch import optim
import pandas as pd
import numpy as np

In [8]:
# load the data
df = pd.read_csv('train.csv')

print(df.tail()) 

# creating tensor from targets_df 
df_x = df.iloc[:,1:]
df_y = df.iloc[:,0]
torch_x = torch.tensor(df_x.values).float()
torch_y = torch.tensor(df_y.values).long()

# Convert to one-hot encoding
num_classes = 10  # Assuming you have 10 classes (0 to 9)
one_hot_encoded = torch.eye(num_classes)[torch_y]

print(one_hot_encoded.shape)

# printing out result
print(torch_x.shape)
print(torch_y.shape)

# split train, val, test set
split = 38000
train_x = torch_x[:split]
train_y = one_hot_encoded[:split]

val_x = torch_x[split:]
val_y = one_hot_encoded[split:]

# printing out result
print(train_x.shape)
print(val_x.shape)

# batching data
batch_size = 32
def train_loader():
    num_batches = train_x.shape[0] // batch_size
    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = (i + 1) * batch_size
        batch_X = train_x[batch_start:batch_end,:]
        batch_Y = val_y[batch_start:batch_end]
        yield batch_X, batch_Y

       label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  \
41995       0  ...         0         0         0         0         0   
41996       0  ...         0         0         0         0         0   
41997       0  ...         0         0         0         0         0   
41998       0  ...         0         0         0         0         0   
41999       0  ...         0         0         0         0         0   

       pixel779  pixel780  pixel781  pixel782  pixel783  
41995         0         0   

In [9]:
# prepare network
model = nn.Sequential(
    nn.Linear(28*28, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

In [10]:
# Define my optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [11]:
# Define loss
loss = nn.CrossEntropyLoss()

In [12]:
# My Training loops
nb_epochs = 5
for epoch in range(nb_epochs):
    i = 0
    for batch in train_loader():
        x, y = batch

        # batch b
        # x 28*28

        # 1. forward
        # print(x[0])
        print(y.shape)
        if y.shape[0] is not 0:
            l = model(x[0])    # l:logits
    
            # 2. compute the objective function
            J = loss(l, y[0])
    
            # 3. cleaning the gradients
            model.zero_grad()
                
            # 4. accumulate the partial derivatives of J wrt params
            J.backward()
    
            # 5. step in hte opposite direction of the gradient
            optimizer.step()
    
            print(f' Epoch {epoch +1}, train loss: {J.item()}')

        i = i+1
    print(i)
        


<>:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_5218/1975751565.py:14: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if y.shape[0] is not 0:


torch.Size([32, 10])
 Epoch 1, train loss: 14.101517677307129
torch.Size([32, 10])
 Epoch 1, train loss: 263.1551208496094
torch.Size([32, 10])
 Epoch 1, train loss: 5897.39501953125
torch.Size([32, 10])
 Epoch 1, train loss: 13772.630859375
torch.Size([32, 10])
 Epoch 1, train loss: 38526.7578125
torch.Size([32, 10])
 Epoch 1, train loss: 28941728.0
torch.Size([32, 10])
 Epoch 1, train loss: 40377344.0
torch.Size([32, 10])
 Epoch 1, train loss: -0.0
torch.Size([32, 10])
 Epoch 1, train loss: 32.462745666503906
torch.Size([32, 10])
 Epoch 1, train loss: 6.313091278076172
torch.Size([32, 10])
 Epoch 1, train loss: 3.2048308849334717
torch.Size([32, 10])
 Epoch 1, train loss: 2.875971794128418
torch.Size([32, 10])
 Epoch 1, train loss: 222.73837280273438
torch.Size([32, 10])
 Epoch 1, train loss: 2.6626062393188477
torch.Size([32, 10])
 Epoch 1, train loss: 1.9185194969177246
torch.Size([32, 10])
 Epoch 1, train loss: 8955814.0
torch.Size([32, 10])
 Epoch 1, train loss: 2.646465063095092